In [43]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os.path
import pickle
from pyhere import here

SCOPES = ["https://mail.google.com/"]  # full access
# https://www.googleapis.com/auth/gmail.readonly
creds = None
if os.path.exists("token.pickle"):
    with open("token.pickle", "rb") as token:
        creds = pickle.load(token)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            here("credentials.json"), SCOPES
        )
        creds = flow.run_local_server(port=0)
    with open("token.pickle", "wb") as token:
        pickle.dump(creds, token)

service = build("gmail", "v1", credentials=creds)
results = (
    service.users()
    .messages()
    .list(userId="me", maxResults=10, q="from:robinhood.com")
    .execute()
)
messages = results.get("messages", [])
for message in messages:
    print(f"Message ID: {message}")

Message ID: {'id': '1956dfdd93839b80', 'threadId': '1956dfdd93839b80'}
Message ID: {'id': '19566a7409a395f6', 'threadId': '19566a7409a395f6'}


In [ ]:
# get the message content
message = service.users().messages().get(userId="me", id=messages[0]["id"]).execute()
print(message)
print(list(message.keys()))

{'id': '1956dfdd93839b80', 'threadId': '1956dfdd93839b80', 'labelIds': ['IMPORTANT', 'CATEGORY_UPDATES', 'INBOX'], 'snippet': 'Hi Wei, Thank you for taking the time to apply for the Senior Data Scientist, Product position. We&#39;ve been extremely fortunate to have a fantastic response from accomplished candidates such as', 'payload': {'partId': '', 'mimeType': 'multipart/alternative', 'filename': '', 'headers': [{'name': 'Delivered-To', 'value': 'weiyang2048@gmail.com'}, {'name': 'Received', 'value': 'by 2002:a05:7301:378e:b0:160:182c:4392 with SMTP id jw14csp876016dyb;        Thu, 6 Mar 2025 16:25:07 -0800 (PST)'}, {'name': 'X-Google-Smtp-Source', 'value': 'AGHT+IHUbFdq024crC1Y0LLdxzaNE/vPjuFeLzMuHfY6a7XTTzOmEfburAZwSj1jYwbCSuzDi9Ub'}, {'name': 'X-Received', 'value': 'by 2002:a05:622a:19a5:b0:474:daf2:a029 with SMTP id d75a77b69052e-47608bcfa7dmr19463801cf.0.1741307107593;        Thu, 06 Mar 2025 16:25:07 -0800 (PST)'}, {'name': 'ARC-Seal', 'value': 'i=1; a=rsa-sha256; t=1741307107; 

In [73]:
import base64
class Message:
    def __init__(self, message):
        self.message = message
        self.snippet = message["snippet"]
        self.get_headers()
        self.body = base64.urlsafe_b64decode(
            message["payload"]["parts"][0]["body"]["data"]
        ).decode("utf-8")

    def get_headers(self):
        self.headers = {
            header["name"]: header["value"]
            for header in self.message["payload"]["headers"]
        }
        self.subject = self.headers["Subject"]
        self.from_ = self.headers["From"]
        self.to = self.headers["To"]
        self.date = self.headers["Date"]
        return self.headers

    def __str__(self):
        return (
            f"From={self.from_},\n"
            f"To={self.to},\n"
            f"Date={self.date},\n"
            f"Subject={self.subject}\n"
            f"snippet={self.snippet}\n"
            f"Body:\n\n{self.body}"
        )
    def delete(self):
        service.users().messages().delete(userId="me", id=self.message["id"]).execute()

    # todo : get what is in the deleted box
m = Message(message)
print(m)
m.delete()
print("---")
m = Message(message)
print(m)
# for key in message.keys():
#     print(key, message[key])
# print("---")

# for key in message["payload"].keys():
#     print(key, message["payload"][key])

# for part in message["payload"]["parts"]:
#     print(part)
# print("---")

# for header in message["payload"]["headers"]:
#     print(header)

# print("---")
# import base64

# # decode the message content
# message_content = message["payload"]["parts"][0]["body"]["data"]
# message_content = base64.urlsafe_b64decode(message_content).decode("utf-8")
# print(message_content)

From=no-reply@robinhood.com,
To=weiyang2048@gmail.com,
Date=Fri, 07 Mar 2025 00:25:06 +0000,
Subject=Important information about your application to Robinhood
snippet=Hi Wei, Thank you for taking the time to apply for the Senior Data Scientist, Product position. We&#39;ve been extremely fortunate to have a fantastic response from accomplished candidates such as
Body:

Hi Wei,

Thank you for taking the time to apply for the Senior Data Scientist, Product position.

We've been extremely fortunate to have a fantastic response from accomplished candidates such as yourself for this role. However, after careful consideration, we've made the decision to not move forward with the interview process at this time. We really appreciate your time and efforts in applying. We’d love to stay in touch as our team continues to grow and reconnect down the line.

Thanks again for your interest in Robinhood, and we wish you the best of luck in your search!

Kind regards,
Robinhood Recruiting Team


HttpError: <HttpError 404 when requesting https://gmail.googleapis.com/gmail/v1/users/me/messages/1956dfdd93839b80? returned "Requested entity was not found.". Details: "[{'message': 'Requested entity was not found.', 'domain': 'global', 'reason': 'notFound'}]">